# TensorTools Runtime Demo

Demonstrates the usage of the runtime using a simple autoencoder model.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
BATCH_SIZE = 24
REG_LAMBDA = 5e-4
NUM_GPUS = 2

In [ ]:
dataset_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(BATCH_SIZE * NUM_GPUS,
                                                                 input_seq_length=1, target_seq_length=0)
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(BATCH_SIZE * NUM_GPUS,
                                                                 input_seq_length=1, target_seq_length=0)
#dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(BATCH_SIZE * NUM_GPUS,
#                                                               input_seq_length=1, target_seq_length=0)

In [ ]:
class SimpleAutoencoderModel(tt.model.AbstractModel):
    """Simple neuronal autoencoder.
    """
    def __init__(self, inputs, targets, reg_lambda,
                 is_training=True, device_scope=None, memory_device=None):
        super(SimpleAutoencoderModel, self).__init__(inputs, targets, reg_lambda,
                                                     is_training, device_scope, memory_device)
    
    @tt.utils.attr.lazy_property
    def predictions(self):
        x = tf.contrib.layers.flatten(self.inputs)
        encoded = tt.network.fc("FC_Enc", x, 128,
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.relu,
                                device=self.memory_device)
        representation = encoded
        decoded = tt.network.fc("FC_Dec", representation, self.input_shape[1] * self.input_shape[2] * self.input_shape[3],
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.sigmoid,
                                device=self.memory_device)
        
        return tf.reshape(decoded, [-1, 1, self.input_shape[2], self.input_shape[3], self.input_shape[4]])

    @tt.utils.attr.lazy_property
    def loss(self):
        return tt.loss.bce(self.predictions, self.targets)

In [ ]:
runtime = tt.core.MultiGpuRuntime()
runtime.register_datasets(dataset_train, dataset_valid)
runtime.register_model(
    lambda x, y, scope, is_training : SimpleAutoencoderModel(x, y,
                                                             reg_lambda=REG_LAMBDA,
                                                             is_training=is_training,
                                                             device_scope=scope))
                # TODO: set memory_device. Change this! (init() method? Factory-method?)
runtime.build(is_autoencoder=True)

In [ ]:
runtime.train(steps=5000)

In [ ]:
runtime.validate()

In [ ]:
runtime.test()

In [ ]:
batch_x, _ = dataset_valid.get_batch()

predictions = runtime.predict(batch_x)
print(predictions.shape)

tt.visualization.display_batch(batch_x[0:4,0,:,:,:])
tt.visualization.display_batch(predictions[0:4,0,:,:,:])

In [ ]:
runtime.close()